## Notes

In [1]:
from dask.distributed import Client

client = Client(processes=True, threads_per_worker=1)
client

KeyboardInterrupt: 

In [1]:
import os

# os.add_dll_directory(
#     "C:\\Users\\ospin\\miniconda3\\envs\\xhydro-dev\\julia_env\\pyjuliapkg\\install\\bin"
# )

# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

# Basic imports
try:
    from xhydro.extreme_value_analysis.parameterestimation import *

    has_julia = True
except ImportError:
    print(
        "This notebook requires the `julia` programming language and the `juliacall` Python package to be installed."
    )
    has_julia = False

import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch
import xarray as xr

# warnings.simplefilter("always", UserWarning)

Cannot find header.dxf (GDAL_DATA is not defined)


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
# ds.chunk({'time': -1, 'station_num': 1}).to_zarr('C:\\Users\\ospin\\OneDrive\\Documents\\Wrapper\\NOAA_GFDL_ESM4_chunks.zarr', mode='w')

ds = xr.open_zarr(
    "C:\\Users\\ospin\\OneDrive\\Documents\\Wrapper\\NOAA_GFDL_ESM4_chunks.zarr"
)
# ds = ds.drop('station_name')
ds

<xarray.Dataset> Size: 455kB
Dimensions:          (station_num: 97, time: 146)
Coordinates:
    station_name     (station_num, time) object 113kB dask.array<chunksize=(97, 146), meta=np.ndarray>
  * station_num      (station_num) int64 776B 1001 1004 1008 ... 15403 15414
  * time             (time) object 1kB '1955-01-01' ... '2100-01-01'
Data variables:
    max_temp_yearly  (station_num, time) float64 113kB dask.array<chunksize=(1, 146), meta=np.ndarray>
    qmax_yearly      (station_num, time) float64 113kB dask.array<chunksize=(1, 146), meta=np.ndarray>
    total_precip     (station_num, time) float64 113kB dask.array<chunksize=(1, 146), meta=np.ndarray>

In [3]:
return_level_quantiles_nstat = fit(
    ds.compute(),
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
)

In [ ]:
return_level_quantiles_nstat.sel(station_num=1001)

In [ ]:
return_level_quantiles_nstat = fit(
    ds,
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    scalecov=["max_temp_yearly"],
    distributed=True,
)

return_level_quantiles_nstat.compute(scheduler="processes")

In [ ]:
params = return_level(
    ds,
    dist="genextreme",
    method="ml",
    vars=["total_precip", "qmax_yearly"],
    confidence_level=0.95,
    locationcov=["max_temp_yearly"],
    distributed=False,
)
params

In [ ]:
params["total_precip"].sel(station_num=1001).compute(scheduler="processes")

In [ ]:
with Client(processes=True, threads_per_worker=1) as executor_dask:
    import dask

    params = fit(
        ds, dist="genextreme", method="ml", vars=["total_precip"], confidence_level=0.95
    )
    with dask.config.set({"multiprocessing.context": "spawn"}):
        r = params["total_precip"].sel(station_num=1001).compute()

        print(r)

In [ ]:
params["total_precip"].sel(station_num=1001).compute()

## Données

In [ ]:
GITHUB_URL = "https://github.com/hydrologie/xhydro-testdata"
BRANCH_OR_COMMIT_HASH = "main"

data = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/extreme_value_analysis/NOAA_GFDL_ESM4.zip",
    known_hash="md5:6c0f51c59add2f37b0e4af169b7eb7f6",
    processor=pooch.Unzip(),
)

df = pd.read_csv(data[0])
ds = df.to_xarray()
ds = ds.set_coords(["time", "station_num", "station_name"])
ds = ds.set_index(index=["station_num", "time"])
ds = ds.unstack("index")
ds

In [ ]:
plt.rcParams["figure.figsize"] = [6.5, 6.5]  # Adjust the figure size
plt.rcParams["figure.autolayout"] = True

# First subplot
plt.subplot(2, 2, 1)  # 2x2 grid, first plot
y = ds.total_precip[0, :].values
x_1 = ds.time.values
plt.title("Précipitation totale à la station Dozois\n selon le temps")
plt.xlabel("Temps")
plt.ylabel("Précipitation totale")
plt.plot(x_1, y, color="red")

# Custom x-axis ticks
first_year = x_1[0]
last_year = x_1[-1]
middle_index = len(x_1) // 2
middle_year = x_1[middle_index]
plt.xticks([first_year, middle_year, last_year])

# Second subplot
plt.subplot(2, 2, 2)  # 2x2 grid, second plot
x_2 = ds.max_temp_yearly[0, :].values
plt.title(
    "Précipitation totale à la station Dozois\n selon la température maximale annuelle"
)
plt.xlabel("Température maximale annuelle")
plt.ylabel("Précipitation totale")
plt.scatter(x_2, y, color="red")

# Third subplot
plt.subplot(2, 2, 3)  # 2x2 grid, third plot
x_3 = ds.qmax_yearly[0, :].values
plt.title("Précipitation totale à la station Dozois\n selon le débit maximal annuel")
plt.xlabel("Débit maximal annuel")
plt.ylabel("Précipitation totale")
plt.scatter(x_3, y, color="red")

# Fourth subplot (empty)
plt.subplot(2, 2, 4)  # 2x2 grid, fourth plot (left empty)
plt.axis("off")  # Turn off the axis

plt.show()

## Estimation de paramètres - Stationnaire


In [ ]:
params = fit(
    ds, dist="genextreme", method="ml", vars=["total_precip"], confidence_level=0.95
)
params

## Estimation de paramètres - Non-Stationnaire

In [3]:
ds_nan = ds.compute()
ds_nan["max_temp_yearly"] = ds_nan["max_temp_yearly"] * np.nan
ds_nan

<xarray.Dataset> Size: 455kB
Dimensions:          (station_num: 97, time: 146)
Coordinates:
    station_name     (station_num, time) object 113kB 'Dozois' ... 'Romaine 4...
  * station_num      (station_num) int64 776B 1001 1004 1008 ... 15403 15414
  * time             (time) object 1kB '1955-01-01' ... '2100-01-01'
Data variables:
    max_temp_yearly  (station_num, time) float64 113kB nan nan nan ... nan nan
    qmax_yearly      (station_num, time) float64 113kB 424.0 805.1 ... 711.7
    total_precip     (station_num, time) float64 113kB 983.6 ... 1.065e+03

In [ ]:
params = return_level(
    ds.compute(),
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    locationcov=["max_temp_yearly"],
)
params

NameError: name 'retunr_level' is not defined

## Estimation de paramètres - Stationnaire: Attention au 0 dans l'intervalle!

In [ ]:
params = fit(
    ds,
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    locationcov=["max_temp_yearly"],
    shapecov=["qmax_yearly"],
    distributed=True,
)
params

## Niveau de retour - Stationnaire

In [ ]:
params = fit(
    ds,
    dist="genpareto",
    method="bayes",
    vars=["total_precip"],
    shapecov=["qmax_yearly"],
    distributed=True,
)
params

In [ ]:
return_level_quantiles_stat = return_level(
    ds,
    dist="genpareto",
    method="bayes",
    vars=["total_precip"],
    return_period=2,
    threshold_pareto=1,
    nobs_pareto=146,
    nobsperblock_pareto=10,
)
return_level_quantiles_stat

In [ ]:
ds

In [ ]:
return_level_quantiles_stat = return_level(
    ds, dist="genextreme", method="ml", vars=["total_precip"], return_period=100
)
return_level_quantiles_stat

## Niveau de retour - Non-Stationnaire

In [ ]:
return_level_quantiles_nstat = return_level(
    ds, dist="genextreme", method="pwm", vars=["total_precip"], return_period=100
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat

In [ ]:
return_level_quantiles_nstat = return_level(
    ds,
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    return_period=100,
    locationcov=["max_temp_yearly"],
    distributed=True,
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat

In [12]:
return_level_quantiles_nstat = return_level(
    ds.sel(station_num=[1001, 1004]).compute(),
    dist="genextreme",
    method="bayes",
    vars=["total_precip"],
    return_period=100,
    locationcov=["max_temp_yearly"],
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat

NameError: name 'nparams' is not defined

In [ ]:
return_level_quantiles_nstat = return_level(
    ds.sel(station_num=[1001, 1004]),
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    return_period=100,
    locationcov=["max_temp_yearly"],
    distributed=True,
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat

In [ ]:
ds2 = ds.sel(station_num=[1001, 1004]).copy()
ds2["max_temp_yearly"].sel(time="2100-01-01")[:] = np.nan

return_level_quantiles_nstat2 = return_level(
    ds2.sel(station_num=[1001, 1004]),
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    return_period=100,
    locationcov=["max_temp_yearly"],
    distributed=True,
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat2

In [ ]:
return_level_quantiles_nstat = return_level(
    ds.sel(station_num=[1001, 1004]),
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    return_period=100,
    distributed=False,
)
# print(return_level_quantiles_nstat.total_precip[0, :].values)
return_level_quantiles_nstat

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Convert string dates to datetime objects
x = pd.to_datetime(ds.time.values)

plt.rcParams["figure.figsize"] = [12, 6.5]
plt.rcParams["figure.autolayout"] = True

y_1 = return_level_quantiles_nstat.total_precip[0, :].values
y_2 = [return_level_quantiles_stat.total_precip[0, :].values for _ in range(len(y_1))]

plt.title(
    "Précipitation totale de récurrence 1:1000 à Dozois - Stationnaire vs Non-Stationnaire"
)
plt.xlabel("Temps")
plt.ylabel("Niveau de retour")

# Custom x-axis ticks
first_year = x[0]
last_year = x[-1]
middle_year = x[len(x) // 2]
plt.xticks(
    [first_year, middle_year, last_year],
    labels=[first_year.year, middle_year.year, last_year.year],
)

# Plot both y_1 and y_2
plt.plot(x, y_1, color="red", label="Niveau de retour non stationnaire")
plt.plot(x, y_2, color="blue", label="Niveau de retour stationnaire")
plt.legend()
plt.show()

# Using dataset rain from Extremes.jl

In [ ]:
import os

os.add_dll_directory(
    "C:\\Users\\ospin\\miniconda3\\envs\\xhydro-dev\\julia_env\\pyjuliapkg\\install\\bin"
)

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Basic imports
try:
    from xhydro.extreme_value_analysis.parameterestimation import *
    from xhydro.extreme_value_analysis.structures.util import *

    has_julia = True
except ImportError:
    print(
        "This notebook requires the `julia` programming language and the `juliacall` Python package to be installed."
    )
    has_julia = False

import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch
import xarray as xr

# warnings.simplefilter("always«««", UserWarning)

In [ ]:
df = pd.read_csv(
    "C:/Users/ospin/OneDrive/Documents/Wrapper/output_data_filter_exced.csv",
    index_col=0,
)

In [ ]:
ds_rain = xr.Dataset.from_dataframe(df)
ds_rain

In [ ]:
return_level(
    ds_rain,
    dist="genpareto",
    method="ML",
    vars=["Rainfall"],
    dim="Date",
    return_period=100,
    threshold_pareto=30.0,
    nobs_pareto=17531,
    nobsperblock_pareto=365,
)

## 1. genpareto

### 1.1 distribution

In [ ]:
fit(
    ds_rain,
    dist="genpareto",
    method="ml",
    vars=["Rainfall"],
    dim="Date",
    confidence_level=0.95,
)

### 1.2 return level

In [ ]:
return_level(
    ds_rain,
    dist="genpareto",
    method="ML",
    vars=["Rainfall"],
    dim="Date",
    return_period=100,
    threshold_pareto=30.0,
    nobs_pareto=17531,
    nobsperblock_pareto=365,
)

In [ ]:
params = fit(
    ds_rain,
    dist="genpareto",
    method="pwm",
    vars=["Rainfall"],
    dim="Date",
    confidence_level=0.95,
)
params

In [ ]:
f(3, 5)

In [ ]:
import numpy as np
import xarray as xr

# Define the data for the DataArray
rainfall_data = np.array([2.0068965, 0.1844927])

# Create the DataArray with the specified dimension
rainfall = xr.DataArray(
    rainfall_data,
    dims="params",  # Dimension name
    name="Rainfall",  # Name of the DataArray
)

# Display the DataArray
print(rainfall)

In [ ]:
rainfall2 = xr.DataArray(
    rainfall_data,
    dims="params3",  # Dimension name
    name="Rainfall2",  # Name of the DataArray
)

In [ ]:
xr.merge(
    [
        rainfall.assign_coords(params=np.arange(2)),
        rainfall2.assign_coords(params3=np.arange(2)),
    ]
)

In [ ]:
dataset = xr.Dataset({"Rainfall": rainfall, "Rainfallw": rainfall})

In [ ]:
params = fit(
    ds_rain,
    dist="genpareto",
    method="ml",
    vars=["Rainfall"],
    dim="Date",
    confidence_level=0.95,
)
params

In [ ]:
d

In [ ]:
d[0]

In [ ]:
exponentiate_logscale(d, [1], [2], pareto=True)

In [ ]:
dataset = xr.Dataset(
    {
        key: (["index"], value)  # Assign data to the corresponding variables
        for key, value in params2.items()
    },
    coords={"index": np.arange(len(params2["params"]))},  # Create an index coordinate
)

In [ ]:
dataset

In [ ]:
xr.DataArray(list(params2.values()), coords=[list(params2.keys())], dims=["keys"])

In [ ]:
return_level(
    ds_rain,
    dist="genpareto",
    method="ML",
    vars=["Rainfall"],
    dim="Date",
    return_period=100,
    threshold_pareto=30.0,
    nobs_pareto=17531,
    nobsperblock_pareto=365,
)

In [ ]:
import numpy as np

In [ ]:
import numpy as np


def create_nan_mask(*nested_lists):
    """
    Create a mask indicating NaN positions across multiple nested lists.

    Parameters:
        *nested_lists: Any number of nested lists (lists of lists).

    Returns:
        A single nested list mask with True where NaNs are present.
    """
    # Convert all input nested lists to numpy arrays
    arrays = [np.array(lst, dtype=float) for lst in nested_lists]
    stack = np.vstack(arrays)
    mask = np.any(np.isnan(stack), axis=0)

    return mask


# Example usage
nested_list1 = [[1, 2, 3, np.nan], [np.nan, 4, 3, 3], [5, 6, 3, 3]]
nested_list2 = [[np.nan, 1, 3, 3], [4, 5, 3, 3]]
nested_list3 = [[3, 4, 3, 2]]

mask = create_nan_mask(nested_list1, nested_list2, nested_list3)
print("NaN Mask:")
print(mask)

In [ ]:
import numpy as np


def create_nan_mask(*nested_lists):
    """
    Create a mask indicating NaN positions across multiple nested lists.

    Parameters:
        *nested_lists: Any number of nested lists (lists of lists).

    Returns:
        A single nested list mask with True where NaNs are present.
    """
    # Validate input
    if not nested_lists:
        raise ValueError("At least one nested list must be provided.")

    # Convert all input nested lists to numpy arrays and stack them
    arrays = [np.array(lst, dtype=float) for lst in nested_lists]
    arrays = [item for sublist in arrays for item in sublist]

    # Stack arrays vertically
    combined_array = np.vstack(arrays)

    # Create a mask for NaN positions
    combined_nan_mask = np.isnan(combined_array)

    # Convert the mask to a nested list format
    mask_nested_list = combined_nan_mask.tolist()

    return mask_nested_list


create_nan_mask(nested_list1, nested_list2, nested_list3)